In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 为自定义训练构建 Vertex AI 试验谱系

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/build_model_experimentation_lineage_with_prebuild_code.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/build_model_experimentation_lineage_with_prebuild_code.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/experiments/build_model_experimentation_lineage_with_prebuild_code.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>

## 概述

作为数据科学家，您希望能够重用团队中其他人编写的代码路径（数据预处理、特征工程等），以简化和标准化所有复杂的数据处理过程。

了解有关[Vertex AI实验](https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments)和[Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata)的更多信息。

### 目标

在这个笔记本中，您将学习如何将预处理代码整合到 Vertex AI 实验中。同时，您将构建实验的谱系，以便记录、分析、调试和审计沿着您的机器学习之旅产生的元数据和工件。

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex ML Metadata
- Vertex AI Experiments

执行的步骤包括：

- 执行用于预处理数据的模块
  - 创建数据集工件
  - 记录参数
- 执行用于训练模型的模块
  - 记录参数
  - 创建模型工件
  - 将跟踪谱系分配给数据集、模型和参数

###数据集

这个数据集是UCI新闻聚合数据集，包含了从2014年3月10日到2014年8月10日期间收集的422,937条新闻。以下是数据集中的示例记录：

|ID |标题                                                                 |网址                                                                                                                  |发布者           |类别|故事                       |主机名           |时间戳        |
|---|---------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------|-----------------|----|---------------------------|-----------------|-------------|
|1  |联储官员表示，天气导致数据疲软，不应减缓削减速度                 |http://www.latimes.com/business/money/la-fi-mo-federal-reserve-plosser-stimulus-economy-20140310,0,1312750.story\?track=rss|洛杉矶时报     |b   |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.latimes.com   |1394470370698|
|2  |联储官员查尔斯·普洛瑟认为缩减速度变化的门槛很高                |http://www.livemint.com/Politics/H2EvwJSK2VE6OF7iK1g3PP/Feds-Charles-Plosser-sees-high-bar-for-change-in-pace-of-ta.html   |Livemint         |b   |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.livemint.com  |1394470371207|
|3  |美股开盘：联储官员暗示加速缩减后股价下跌                      |http://www.ifamagazine.com/news/us-open-stocks-fall-after-fed-official-hints-at-accelerated-tapering-294436                |IFA Magazine     |b   |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.ifamagazine.com|1394470371550|
|4  |联储官员查尔斯·普洛瑟表示，联储面临“落后曲线”的风险          |http://www.ifamagazine.com/news/fed-risks-falling-behind-the-curve-charles-plosser-says-294430                             |IFA Magazine     |b   |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.ifamagazine.com|1394470371793|
|5  |联储普洛瑟：恶劣天气抑制了就业增长                           |http://www.moneynews.com/Economy/federal-reserve-charles-plosser-weather-job-growth/2014/03/10/id/557011                   |Moneynews        |b   |ddUyU0VZz0BRneMioxUPQVP6sIxvM|www.moneynews.com|1394470372027|

### 费用

本教程使用谷歌云的可计费组件：

* Vertex AI
* 云存储

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[云存储 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

### 安装额外的软件包

在您的笔记本环境中安装未安装的额外软件包依赖项。

In [ ]:
! pip3 install --upgrade --quiet joblib fsspec gcsfs scikit-learn 
! pip3 install --upgrade --quiet google-cloud-aiplatform==1.35 

只有Colab：请取消下面的单元格注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您也可以更改由Vertex AI使用的`REGION`变量。了解有关[ Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud帐户

根据您的Jupyter环境，您可能需要手动进行验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench**
* 因为已经通过验证，所以无需采取任何措施。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作、取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

请参考如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples为您的服务帐号授予Cloud Storage权限。

### 创建一个云存储桶

创建一个存储桶来存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚未存在时，才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

### 设置数据文件夹

In [ ]:
DATA_PATH = "data"
! mkdir -m 777 -p {DATA_PATH}

获取数据

In [ ]:
DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip"
! wget --no-parent {DATASET_URL} --directory-prefix={DATA_PATH}
! mkdir -m 777 -p {DATA_PATH}/temp {DATA_PATH}/raw
! unzip {DATA_PATH}/*.zip -d {DATA_PATH}/temp
! mv {DATA_PATH}/temp/*.csv {DATA_PATH}/raw && rm -Rf {DATA_PATH}/temp && rm -f {DATA_PATH}/*.zip

! gsutil -m cp -R {DATA_PATH}/raw $BUCKET_URI/{DATA_PATH}/raw

### 导入库

In [ ]:
# General
import logging

logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

import collections
import tempfile
import time
import uuid
from json import dumps

collections.Iterable = collections.abc.Iterable

# Vertex AI
from google.cloud import aiplatform as vertex_ai

### 定义常量

In [ ]:
import os

# Base
DATASET_NAME = "news_corpora"
DATASET_URI = f"{BUCKET_URI}/{DATA_PATH}/raw/newsCorpora.csv"

# Experiments
TASK = "classification"
MODEL_TYPE = "naivebayes"
EXPERIMENT_NAME = f"{TASK}-{MODEL_TYPE}-{uuid.uuid1()}"
EXPERIMENT_RUN_NAME = "run-1"

# Preprocessing
PREPROCESSED_DATASET_NAME = f"preprocessed_{DATASET_NAME}"
PREPROCESS_EXECUTION_NAME = "preprocess"
COLUMN_NAMES = [
    "id",
    "title",
    "url",
    "publisher",
    "category",
    "story",
    "hostname",
    "timestamp",
]
DELIMITER = "	"
INDEX_COL = 0
PREPROCESSED_DATASET_URI = (
    f"{BUCKET_URI}/{DATA_PATH}/preprocess/{PREPROCESSED_DATASET_NAME}.csv"
)

# Training
TRAIN_EXECUTION_NAME = "train"
TARGET = "category"
TARGET_LABELS = ["b", "t", "e", "m"]
FEATURES = "title"
TEST_SIZE = 0.2
SEED = 8
TRAINED_MODEL_URI = f"{BUCKET_URI}/deliverables/{uuid.uuid1()}"
MODEL_NAME = f"{EXPERIMENT_NAME}-model"

### 初始化 Python 顶点 AI SDK

为您的项目和对应的存储桶初始化 Python 顶点 AI SDK。

In [ ]:
vertex_ai.init(
    project=PROJECT_ID, experiment=EXPERIMENT_NAME, staging_bucket=BUCKET_URI
)

设置预构建的容器

设置用于训练和预测的预构建 Docker 容器映像。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
SERVE_IMAGE = vertex_ai.helpers.get_prebuilt_prediction_container_uri(
    framework="sklearn", framework_version="1.0", accelerator="cpu"
)

初始化实验运行

In [ ]:
run = vertex_ai.start_run(EXPERIMENT_RUN_NAME)

使用预先构建的数据预处理代码进行模型实验

### 数据预处理

在这一步中，您对原始数据进行一些预处理，以创建训练数据集。

事实上，您可能会遇到一些数据预处理代码，可能是由团队中的其他人编写的。因此，您需要一种方法，在实验运行中集成预处理代码，以标准化和重复使用您所处理的所有复杂数据整理操作。

使用Vertex AI Experiments，您可以通过在实验上下文中添加一个 `with` 语句来跟踪该代码作为运行执行的一部分。

创建数据集元数据工件 

首先，您需要创建数据集元数据工件来追踪 Vertex ML Metadata 中的数据集资源，并创建实验谱系。

In [ ]:
raw_dataset_artifact = vertex_ai.Artifact.create(
    schema_title="system.Dataset", display_name=DATASET_NAME, uri=DATASET_URI
)

#### 创建一个预处理模块

接下来，您将构建一个简单的预处理模块，用于转换文本大小写并删除标点符号。

In [ ]:
"""
Preprocess module
"""


import string

import pandas as pd


def preprocess(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
    """
    Preprocess text
    Args:
        df: The DataFrame to preprocesss
        text_col: The text column name
    Returns:
        preprocessed_df: The datafrane with text in lowercase and without punctuation
    """
    preprocessed_df = df.copy()
    preprocessed_df[text_col] = preprocessed_df[text_col].apply(lambda x: x.lower())
    preprocessed_df[text_col] = preprocessed_df[text_col].apply(
        lambda x: x.translate(str.maketrans("", "", string.punctuation))
    )
    return preprocessed_df

#### 添加“预处理” 执行

Vertex AI实验支持跟踪执行和工件。 执行是ML工作流程中的步骤，可以包括但不限于数据预处理、训练和模型评估。 执行可以消耗数据集等工件，产生模型等工件。

您可以添加预处理步骤以跟踪其在与Vertex AI实验相关的谱系中的执行。
对于Vertex AI，参数是通过消息字段传递的，我们可以在日志中看到。 这些日志的结构是预定义的。

In [ ]:
with vertex_ai.start_execution(
    schema_title="system.ContainerExecution", display_name=PREPROCESS_EXECUTION_NAME
) as exc:
    logging.info(f"Start {PREPROCESS_EXECUTION_NAME} execution.")
    exc.assign_input_artifacts([raw_dataset_artifact])

    # Log preprocessing params --------------------------------------------------
    logging.info("Log preprocessing params.")
    vertex_ai.log_params(
        {
            "delimiter": DELIMITER,
            "features": dumps(COLUMN_NAMES),
            "index_col": INDEX_COL,
        }
    )

    # Preprocessing ------------------------------------------------------------
    logging.info("Preprocessing.")
    raw_df = pd.read_csv(
        raw_dataset_artifact.uri,
        delimiter=DELIMITER,
        names=COLUMN_NAMES,
        index_col=INDEX_COL,
    )
    preprocessed_df = preprocess(raw_df, "title")
    preprocessed_df.to_csv(PREPROCESSED_DATASET_URI, sep=",")

    # Log preprocessing metrics and store dataset artifact ---------------------
    logging.info(f"Log preprocessing metrics and {PREPROCESSED_DATASET_NAME} dataset.")
    vertex_ai.log_metrics(
        {
            "n_records": preprocessed_df.shape[0],
            "n_columns": preprocessed_df.shape[1],
        },
    )

    preprocessed_dataset_metadata = vertex_ai.Artifact.create(
        schema_title="system.Dataset",
        display_name=PREPROCESSED_DATASET_NAME,
        uri=PREPROCESSED_DATASET_URI,
    )
    exc.assign_output_artifacts([preprocessed_dataset_metadata])

### 模型训练

在这一步中，您将训练一个多项式朴素贝叶斯管道。

#### 创建模型训练模块

在训练模块下面。

**get_training_split：**它接受参数如 x（要拆分的数据）、y（要拆分的标签）、test_size（保留用于测试的数据比例）和 random_state（随机数生成器使用的种子）。
该函数返回训练数据、测试数据、训练标签和测试标签。

**get_pipeline：**它返回模型。

**train_pipeline：**它使用模型、训练数据、训练标签训练模型并返回经过训练的模型。

**evaluate_model：**它评估模型并返回模型的准确度。

In [ ]:
"""
Train module
"""

import joblib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import (accuracy_score, confusion_matrix, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


def get_training_split(
    x: pd.DataFrame, y: pd.Series, test_size: float, random_state: int
) -> (pd.DataFrame, pd.Series, pd.DataFrame, pd.Series):
    """
    Splits data into training and testing sets
    Args:
        x: The data to be split
        y: The labels to be split
        test_size: The proportion of the data to be reserved for testing
        random_state: The seed used by the random number generator
    Returns:
        x_train: The training data
        x_test: The testing data
        y_train: The training labels
        y_test: The testing labels
    """

    x_train, x_val, y_train, y_val = train_test_split(
        x, y, test_size=test_size, random_state=random_state
    )
    return x_train, x_val, y_train, y_val


def get_pipeline():
    """
    Get the model
    Args:
        None
    Returns:
        model: The model
    """
    model = Pipeline(
        [
            ("vect", CountVectorizer()),
            ("tfidf", TfidfTransformer()),
            ("clf", MultinomialNB()),
        ]
    )
    return model


def train_pipeline(model: Pipeline, X_train: pd.Series, y_train: pd.Series) -> Pipeline:
    """
    Train the model
    Args:
        model: The model to train
        X_train: The training data
        y_train: The training labels
    Returns:
        model: The trained model
    """
    model.fit(X_train, y_train)
    return model


def evaluate_model(model: Pipeline, X_test: pd.Series, y_test: pd.Series) -> float:
    """
    Evaluate the model
    Args:
        model: The model to evaluate
        X_test: The testing data
        y_test: The testing labels
    Returns:
        score: The accuracy of the model
    """
    # Evaluate model
    y_pred = model.predict(X_test)

    # Store evaluation metrics
    summary_metrics = {
        "accuracy": round(accuracy_score(y_test, y_pred), 5),
        "precision": round(precision_score(y_test, y_pred, average="weighted"), 5),
        "recall": round(recall_score(y_test, y_pred, average="weighted"), 5),
    }
    classification_metrics = {
        "matrix": confusion_matrix(y_test, y_pred, labels=TARGET_LABELS).tolist(),
        "labels": TARGET_LABELS,
    }

    return summary_metrics, classification_metrics


def save_model(model: Pipeline, save_path: str) -> int:
    try:
        with tempfile.NamedTemporaryFile() as tmp:
            joblib.dump(trained_pipeline, filename=tmp.name)
            ! gsutil cp {tmp.name} {save_path}/model.joblib
    except RuntimeError as error:
        print(error)
    return 1

接下来，您需要将训练任务添加到实验执行中，以更新实验谱系。

In [ ]:
with vertex_ai.start_execution(
    schema_title="system.ContainerExecution", display_name=TRAIN_EXECUTION_NAME
) as exc:

    exc.assign_input_artifacts([preprocessed_dataset_metadata])

    # Get training and testing data
    logging.info("Get training and testing data.")
    x_train, x_val, y_train, y_val = get_training_split(
        preprocessed_df[FEATURES],
        preprocessed_df[TARGET],
        test_size=TEST_SIZE,
        random_state=SEED,
    )
    # Get model pipeline
    logging.info("Get model pipeline.")
    pipeline = get_pipeline()

    # Log training param -------------------------------------------------------

    # Log data parameters
    logging.info("Log data parameters.")
    vertex_ai.log_params(
        {
            "target": TARGET,
            "features": FEATURES,
            "test_size": TEST_SIZE,
            "random_state": SEED,
        }
    )

    # Log pipeline parameters
    logging.info("Log pipeline parameters.")
    vertex_ai.log_params(
        {
            "pipeline_steps": dumps(
                {step[0]: str(step[1].__class__.__name__) for step in pipeline.steps}
            )
        }
    )

    # Training -----------------------------------------------------------------

    # Train model pipeline
    logging.info("Train model pipeline.")
    train_start = time.time()
    trained_pipeline = train_pipeline(pipeline, x_train, y_train)
    train_end = time.time()

    # Evaluate model
    logging.info("Evaluate model.")
    summary_metrics, classification_metrics = evaluate_model(
        trained_pipeline, x_val, y_val
    )

    # Log training metrics and store model artifact ----------------------------

    # Log training metrics
    logging.info("Log training metrics.")
    vertex_ai.log_metrics(summary_metrics)
    vertex_ai.log_classification_metrics(
        labels=classification_metrics["labels"],
        matrix=classification_metrics["matrix"],
        display_name="my-confusion-matrix",
    )

    # Generate first ten predictions
    logging.info("Generate prediction sample.")
    prediction_sample = trained_pipeline.predict(x_val)[:10]
    print("prediction sample:", prediction_sample)

    # Upload Model on Vertex AI
    logging.info("Upload Model on Vertex AI.")
    loaded = save_model(trained_pipeline, TRAINED_MODEL_URI)
    if loaded:
        model = vertex_ai.Model.upload(
            serving_container_image_uri=SERVE_IMAGE,
            artifact_uri=TRAINED_MODEL_URI,
            display_name=MODEL_NAME,
        )

    exc.assign_output_artifacts([model])

停止实验运行

In [ ]:
run.end_run()

### 可视化实验谱系

以下是 Vertex AI 元数据 UI 在控制台中显示实验谱系的链接。

In [ ]:
print("Open the following link:", exc.get_output_artifacts()[0].lineage_console_uri)

清理

要清理此项目中使用的所有 Google Cloud 资源，您可以 [删除您用于本教程的 Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的个别资源。

In [ ]:
# Delete experiment
exp = vertex_ai.Experiment(EXPERIMENT_NAME)
exp.delete()

# Delete model
model_list = vertex_ai.Model.list(filter=f'display_name="{MODEL_NAME}"')
for model in model_list:
    model.delete()

# Delete dataset
for dataset_name in [DATASET_NAME, PREPROCESSED_DATASET_NAME]:
    dataset_list = vertex_ai.TabularDataset.list(
        filter=f'display_name="{dataset_name}"'
    )
    for dataset in dataset_list:
        dataset.delete()

# Delete Cloud Storage objects that were created
delete_bucket = True

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI

!rm -Rf {DATA_PATH}